In [130]:
import numpy as np
import matplotlib.pyplot as plt
import poppy
import astropy.units as u

In [131]:
import os


def calc_defocus_coefficient(d_obj, d_focus, f_mm=35.0, f_number=2.8):
    """ 
    Make defocus Zernike coefficient for given object and focus distances.
    d_obj: object distance in meters
    d_focus: focus distance in meters
    f_mm: focal length in mm (default is 35.0)
    f_number: f-number (default is 2.8)
    """
    # deal with infinite object distance
    inv_d_obj = 0.0 if d_obj == np.inf else 1.0/d_obj
    inv_d_focus = 1.0 / d_focus

    # Diopter Difference
    delta_diopter = inv_d_obj - inv_d_focus  

    # Change in wavefront curvature
    f_m = f_mm * 1e-3  # focal length in meters
    w_pv = (f_m**2 / (8.0 * f_number**2)) * delta_diopter  # in meters

    return w_pv

In [ ]:

def generate_poppy_psf(
    d_obj, 
    d_focus, 
    zernike_coeffs=None, 
    wavelength=550e-9, 
    pixel_size=3.45e-6, 
    fov_pixels=128
):
    # --- 1. 硬件参数定义 ---
    f_mm = 25.0 * u.mm      # 焦距 25mm
    f_number = 2.8          # 光圈 F2.8
    
    # 入瞳直径 D = f / F#
    pupil_diameter = f_mm / f_number
    
    # --- 2. 构建光学系统 ---
    osys = poppy.OpticalSystem()
    
    # 入瞳 (圆形)
    osys.add_pupil(poppy.CircularAperture(radius=pupil_diameter/2))
    
    # --- 3. 处理 Zernike 系数 (修复 TypeError) ---
    if zernike_coeffs is None:
        zernike_coeffs = {}
        
    # 计算物理离焦并叠加 (注意：这里 calc_defocus_coefficient 返回的是纯数值，单位米)
    # 我们需要确保 calc_defocus_coefficient 函数在你之前的代码块中已经定义好
    defocus_w = calc_defocus_coefficient(d_obj, d_focus)
    
    # 更新字典
    current_defocus = zernike_coeffs.get(4, 0.0)
    zernike_coeffs[4] = current_defocus + defocus_w
    
    # 将字典转换为列表 (关键修复！)
    # poppy 需要一个列表 [Z1, Z2, Z3, ...]，而不是字典
    max_noll = max(zernike_coeffs.keys()) if zernike_coeffs else 0
    # 至少需要长度为 4 (包含 Z4 Defocus)
    list_len = max(max_noll, 4) 
    coeffs_list = [0.0] * list_len
    
    for noll_idx, val in zernike_coeffs.items():
        if noll_idx > 0:
            coeffs_list[noll_idx - 1] = val # Noll Index 1-based -> List Index 0-based
            
    # 添加像差
    osys.add_pupil(poppy.ZernikeWFE(
        coefficients=coeffs_list, 
        radius=pupil_diameter/2
    ))
    
    # --- 4. 定义探测器 (修复 UnitConversionError) ---
    # 将物理像素尺寸 (m) 转换为角像素尺寸 (arcsec)
    # theta ~ x / f
    pixel_scale_rad = (pixel_size * u.meter) / f_mm
    pixel_scale_arcsec = pixel_scale_rad.to(u.arcsec, equivalencies=u.dimensionless_angles())
    
    osys.add_detector(
        pixelscale=pixel_scale_arcsec / u.pixel, 
        fov_pixels=fov_pixels
    )
    
    # --- 5. 计算 PSF ---
    psf_hdu = osys.calc_psf(wavelength=wavelength, display_intermediates=False)
    psf_image = psf_hdu[0].data 
    
    return psf_image

In [133]:
def demo_poppy_simulation():
    """
    Demo function to generate and plot PSF using poppy.
    """
    d_obj = 1.5
    focus_near = 0.7  # 0 degree focus
    focus_far = 5.0  # 90 degree focus

    optimized_coeffs = {11: 200e-9}

    print(f"Simulating Object Distance at {d_obj}m ..")
    psf_near = generate_poppy_psf(d_obj, focus_near, zernike_coeffs=optimized_coeffs)

    print(f"Simulating Object Distance at {d_obj}m ..")
    psf_far = generate_poppy_psf(d_obj, focus_far, zernike_coeffs=optimized_coeffs)

    # Plot PSFs
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title(f'PSF at Focus {focus_near}m\n@ Object {d_obj}m')
    plt.imshow(psf_near**0.5, cmap='inferno', origin='lower')
    plt.colorbar(label='Intensity (Sqrt)')

    plt.subplot(1, 2, 2)
    plt.title(f'PSF at Focus {focus_far}m\n@ Object {d_obj}m')
    plt.imshow(psf_far**0.5, cmap='inferno', origin='lower')
    plt.colorbar(label='Intensity (Sqrt)')

    plt.tight_layout()
    plt.show()

In [134]:
demo_poppy_simulation()

Simulating Object Distance at 1.5m ..


UnitConversionError: 'm' (length) and 'arcsec' (angle) are not convertible